In [ ]:
import matplotlib.pyplot as plt
import h5py
import rotation_mio as rot
import numpy as np
import time_conversion as time
import z50 as z50_prof

ID = np.loadtxt('../_data/my_halos.dat')[:,0]

path = '/store/erebos/omarioni/_simulations/snap_127/'

for i in range(14):

    snap = h5py.File(path + 'subhalo_'+str('%d'%ID[i])+'.h5py', 'r')

    print('subhalo_'+str('%d'%ID[i]))

    aexp = snap['/Time'][()]
    h    = snap['/h'][()]
    Rvir = snap['/R200'][()]
    Om_L = snap['/Omega_Lambda'][()]
    Om_M = snap['/Omega_0'][()]

    pstr = snap['/Str/Coordinates'][()]
    mstr = snap['/Str/Masses'][()]
    vstr = snap['/Str/Velocities'][()]

    # met  = snap['/Str/Total_Metallicity'][()]
    sft  = snap['/Str/FormationTime'][()]

    amask, = np.where(sft>0) #aca puede haber particulas con age<0, esas no son estrellas son gas cells.
    ages = time.conv2(sft[amask], h, Om_L, Om_M) #transformo el tiempo de formacion de factor de escala a Gyr
    age  = np.max(ages) - ages #lo resto asi saco la edad de las particulas (inverso al tiempo de form)

    # Ztot = met[amask]/0.0127 #para pasarlo a metalicidad solar (ver Illustris data)

    xstr = pstr[amask,0]*aexp/h
    ystr = pstr[amask,1]*aexp/h
    zstr = pstr[amask,2]*aexp/h
    rstr = np.sqrt(xstr**2+ystr**2+zstr**2)

    v_x = vstr[amask,0] *np.sqrt(aexp)
    v_y = vstr[amask,1] *np.sqrt(aexp)
    v_z = vstr[amask,2] *np.sqrt(aexp)

    #----------------------masas----------------------------
    mstr = mstr[amask]*1e10/h

    rgal = 0.15*Rvir*aexp/h

    limit, = np.where(rstr < rgal)
    rsort = np.argsort(rstr[limit])
    Mc = np.cumsum(mstr[limit][rsort])
    Mgal = Mc[-1]
    limit50, = np.where(Mc < Mgal/2)
    r50 = rstr[limit][rsort][limit50][-1]

    veloc,=np.where(rstr < r50/2.)

    #----------componentes de la velocidad del centro de masa------------
    vxcm = sum(mstr[veloc]*v_x[veloc])/sum(mstr[veloc])
    vycm = sum(mstr[veloc]*v_y[veloc])/sum(mstr[veloc])
    vzcm = sum(mstr[veloc]*v_z[veloc])/sum(mstr[veloc])

    vx = v_x - vxcm
    vy = v_y - vycm
    vz = v_z - vzcm

    e1x,e2x,e3x,e1y,e2y,e3y,e1z,e2z,e3z = rot.rot1(mstr,xstr,ystr,zstr,vx,vy,vz,r50/2.)

    xn_str = e1x*xstr + e1y*ystr + e1z*zstr
    yn_str = e2x*xstr + e2y*ystr + e2z*zstr
    zn_str = e3x*xstr + e3y*ystr + e3z*zstr
    vxn = e1x*vx + e1y*vy + e1z*vz
    vyn = e2x*vx + e2y*vy + e2z*vz
    vzn = e3x*vx + e3y*vy + e3z*vz
    
    rn_str = np.sqrt(xn_str**2 + yn_str**2 + zn_str**2)

    Id = ''
    for aux in list(str('%d'%ID[i]))[-2:]:
        Id += aux
        
    gal, = np.where(rn_str < rgal)
    
    # vx_mean = np.mean(vxn[gal])
    # vy_mean = np.mean(vyn[gal])
    vz_mean = np.mean(vzn[gal])
    
#     vx_sqdiff = (vxn[gal] - vx_mean)**2
#     vx_disp2 = np.sum(vx_sqdiff)/(len(vxn[gal])-1)
#     vx_disp = np.sqrt(vx_disp2)

#     vy_sqdiff = (vyn[gal] - vy_mean)**2
#     vy_disp2 = np.sum(vy_sqdiff)/(len(vyn[gal])-1)
#     vy_disp = np.sqrt(vy_disp2)

    vz_sqdiff = (vzn[gal] - vz_mean)**2
    vz_disp2 = np.sum(vz_sqdiff)/(len(vzn[gal])-1)
    vz_disp = np.sqrt(vz_disp2)
    
    fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(8, 8), sharex=True)
    fig.subplots_adjust(bottom=0.10, left =0.12, right = 0.96, top = 0.92,hspace=0.02)
    
   
    ax.minorticks_on()
    ax.tick_params( labelsize=22)
    ax.tick_params('both', length=7, width=1.2,which='minor', direction='in', right='on',top='on')
    ax.tick_params('both', length=10, width=1.2,which='major', direction='in', right='on',top='on')
    ax.set_ylabel('z [kpc]',fontsize=22)
    ax.set_xlabel('R [kpc]',fontsize=22)

    plt.show()

    # fig.savefig('../_imagenes/metallicity_ages/Ztot_ages_corect_s'+str('%d'%ID[i])+'.png',dpi=75, xxbox_inches='tight')